In [1]:
import urllib3
from bs4 import BeautifulSoup
import requests
import pandas as pd

# Step-1: Web Scraping

In [2]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36 Edg/122.0.0.0'
}

In [3]:
url = 'https://www.imdb.com/chart/tvmeter/?ref_=nv_tvv_mptv' # data fetched on: 10th March, 2024
response = requests.get(url, headers = headers)
soup = BeautifulSoup(response.content,'html.parser')

In [4]:
response

<Response [200]>

In [5]:
rank = []
show = []

year = []
eps = []
tv_grade = []

yet = []
rating = []
tv_type = []
season = []

In [6]:
movie_data = soup.findAll('li',attrs = {'class': 'ipc-metadata-list-summary-item sc-10233bc-0 iherUv cli-parent'})

In [7]:
for movie in movie_data:
    
    rank_ = movie.find('div', class_='sc-b8b74125-0 eukXAN meter-const-ranking sc-b0691f29-6 jDMAfU cli-meter-title-header').text
    rank.append(rank_)
    
    name = movie.find('div', class_='ipc-title ipc-title--base ipc-title--title ipc-title-link-no-icon ipc-title--on-textPrimary sc-b0691f29-9 klOwFB cli-title').h3.text
    show.append(name)
    
    rating_ = movie.find('div', class_='sc-e2dbc1a3-0 ajrIH sc-b0691f29-2 bhhtyj cli-ratings-container').span.text
    rating.append(rating_)
    
    tv_type_ = movie.find('span', class_='sc-b0691f29-3 iobobZ cli-title-type-data').text
    tv_type.append(tv_type_)

In [8]:
for movie in movie_data:
    year_name = movie.findAll('div', class_='sc-b0691f29-7 hrgukm cli-title-metadata')
    html_code = str(year_name[0])
    soup = BeautifulSoup(html_code, 'html.parser')
    a=soup.findAll('span', class_='sc-be6f1408-8 fcCUPU cli-title-metadata-item')
    
    years_span = soup.find('span', class_='sc-b0691f29-8 ilsLEX cli-title-metadata-item', string=True)
    years = [years_span.string] if years_span else []
    year.append(years)
    
    episode_count_span = years_span.find_next('span', class_='sc-b0691f29-8 ilsLEX cli-title-metadata-item', string=True)
    episode_count = [episode_count_span.string] if episode_count_span else []
    eps.append(episode_count)
    
    grade_span = episode_count_span.find_next('span', class_='sc-b0691f29-8 ilsLEX cli-title-metadata-item', string=True)
    grade = [grade_span.string] if grade_span else []
    tv_grade.append(grade)

In [9]:
df = pd.DataFrame(list(zip(rank,show,year,eps,rating,tv_type,tv_grade)), columns = ['Rank','Show_name','Year','Episodes','Rating','Type','Grade'])
df

,Rank,Show_name,Year,Episodes,Rating,Type,Grade
0,1 (1),Avatar: The Last Airbender,[2024– ],[10 eps],7.3 (50K),TV Series,[TV-PG]
1,2 (22),Shōgun,[2024],[10 eps],9.2 (22K),TV Mini Series,[TV-14]
2,3 (2),True Detective,[2014– ],[31 eps],8.9 (648K),TV Series,[A]
3,4 (39),The Walking Dead: The Ones Who Live,[2024],[6 eps],8.9 (11K),TV Series,[TV-MA]
4,5 (2),One Day,[2024],[14 eps],8.2 (29K),TV Mini Series,[TV-MA]
...,...,...,...,...,...,...,...
95,96 (15),Monk,[2002–2009],[124 eps],8.1 (84K),TV Series,[TV-PG]
96,97 (47),The Gentlemen,[2024– ],[8 eps],8.4 (6.3K),TV Series,[TV-MA]
97,98 (21),Bridgerton,[2020– ],[25 eps],7.4 (166K),TV Series,[18]
98,99 (6),Love Is Blind,[2020– ],[83 eps],6.2 (11K),TV Series,[16]


In [10]:
df.to_csv('imdb.csv')

# Step-2: Data Wrangling

In [56]:
df = pd.read_csv('imdb.csv')
df.drop('Unnamed: 0', axis=1, inplace=True)
df.head()

,Rank,Show_name,Year,Episodes,Rating,Type,Grade
0,1 (1),Avatar: The Last Airbender,['2024– '],['10 eps'],7.3 (50K),TV Series,['TV-PG']
1,2 (22),Shōgun,['2024'],['10 eps'],9.2 (22K),TV Mini Series,['TV-14']
2,3 (2),True Detective,['2014– '],['31 eps'],8.9 (648K),TV Series,['A']
3,4 (39),The Walking Dead: The Ones Who Live,['2024'],['6 eps'],8.9 (11K),TV Series,['TV-MA']
4,5 (2),One Day,['2024'],['14 eps'],8.2 (29K),TV Mini Series,['TV-MA']


In [57]:
df.isnull().sum()

Rank         0
Show_name    0
Year         0
Episodes     0
Rating       0
Type         0
Grade        0
dtype: int64

In [58]:
df.duplicated().sum()

0

In [59]:
def g(text):
    return int(text.split()[0])

df['Rank'] = df['Rank'].apply(g)
df

,Rank,Show_name,Year,Episodes,Rating,Type,Grade
0,1,Avatar: The Last Airbender,['2024– '],['10 eps'],7.3 (50K),TV Series,['TV-PG']
1,2,Shōgun,['2024'],['10 eps'],9.2 (22K),TV Mini Series,['TV-14']
2,3,True Detective,['2014– '],['31 eps'],8.9 (648K),TV Series,['A']
3,4,The Walking Dead: The Ones Who Live,['2024'],['6 eps'],8.9 (11K),TV Series,['TV-MA']
4,5,One Day,['2024'],['14 eps'],8.2 (29K),TV Mini Series,['TV-MA']
...,...,...,...,...,...,...,...
95,96,Monk,['2002–2009'],['124 eps'],8.1 (84K),TV Series,['TV-PG']
96,97,The Gentlemen,['2024– '],['8 eps'],8.4 (6.3K),TV Series,['TV-MA']
97,98,Bridgerton,['2020– '],['25 eps'],7.4 (166K),TV Series,['18']
98,99,Love Is Blind,['2020– '],['83 eps'],6.2 (11K),TV Series,['16']


In [60]:
def extract_range(text, start, end):
    return text[start:end]

df['Year'] = df['Year'].apply(lambda x: extract_range(x, 2, 6))
df

,Rank,Show_name,Year,Episodes,Rating,Type,Grade
0,1,Avatar: The Last Airbender,2024,['10 eps'],7.3 (50K),TV Series,['TV-PG']
1,2,Shōgun,2024,['10 eps'],9.2 (22K),TV Mini Series,['TV-14']
2,3,True Detective,2014,['31 eps'],8.9 (648K),TV Series,['A']
3,4,The Walking Dead: The Ones Who Live,2024,['6 eps'],8.9 (11K),TV Series,['TV-MA']
4,5,One Day,2024,['14 eps'],8.2 (29K),TV Mini Series,['TV-MA']
...,...,...,...,...,...,...,...
95,96,Monk,2002,['124 eps'],8.1 (84K),TV Series,['TV-PG']
96,97,The Gentlemen,2024,['8 eps'],8.4 (6.3K),TV Series,['TV-MA']
97,98,Bridgerton,2020,['25 eps'],7.4 (166K),TV Series,['18']
98,99,Love Is Blind,2020,['83 eps'],6.2 (11K),TV Series,['16']


In [61]:
df['Episodes'] = df['Episodes'].apply(lambda x: extract_range(x, 2, 4))
df

,Rank,Show_name,Year,Episodes,Rating,Type,Grade
0,1,Avatar: The Last Airbender,2024,10,7.3 (50K),TV Series,['TV-PG']
1,2,Shōgun,2024,10,9.2 (22K),TV Mini Series,['TV-14']
2,3,True Detective,2014,31,8.9 (648K),TV Series,['A']
3,4,The Walking Dead: The Ones Who Live,2024,6,8.9 (11K),TV Series,['TV-MA']
4,5,One Day,2024,14,8.2 (29K),TV Mini Series,['TV-MA']
...,...,...,...,...,...,...,...
95,96,Monk,2002,12,8.1 (84K),TV Series,['TV-PG']
96,97,The Gentlemen,2024,8,8.4 (6.3K),TV Series,['TV-MA']
97,98,Bridgerton,2020,25,7.4 (166K),TV Series,['18']
98,99,Love Is Blind,2020,83,6.2 (11K),TV Series,['16']


In [62]:
df['Rating_rate'] = df['Rating'].apply(lambda x: extract_range(x, 0, 3))
df

,Rank,Show_name,Year,Episodes,Rating,Type,Grade,Rating_rate
0,1,Avatar: The Last Airbender,2024,10,7.3 (50K),TV Series,['TV-PG'],7.3
1,2,Shōgun,2024,10,9.2 (22K),TV Mini Series,['TV-14'],9.2
2,3,True Detective,2014,31,8.9 (648K),TV Series,['A'],8.9
3,4,The Walking Dead: The Ones Who Live,2024,6,8.9 (11K),TV Series,['TV-MA'],8.9
4,5,One Day,2024,14,8.2 (29K),TV Mini Series,['TV-MA'],8.2
...,...,...,...,...,...,...,...,...
95,96,Monk,2002,12,8.1 (84K),TV Series,['TV-PG'],8.1
96,97,The Gentlemen,2024,8,8.4 (6.3K),TV Series,['TV-MA'],8.4
97,98,Bridgerton,2020,25,7.4 (166K),TV Series,['18'],7.4
98,99,Love Is Blind,2020,83,6.2 (11K),TV Series,['16'],6.2


In [65]:
import re
def extract_rating(text):
    match = re.search(r"\d+K", text)
    return match.group() if match else None

df['Rating_count'] = df['Rating'].apply(extract_rating)
df['Rating_count'] = pd.to_numeric(df['Rating_count'].str.replace('K', ''))*1000
df.drop('Rating', axis=1, inplace=True)
df

,Rank,Show_name,Year,Episodes,Type,Grade,Rating_rate,Rating_count
0,1,Avatar: The Last Airbender,2024,10,TV Series,['TV-PG'],7.3,50000.0
1,2,Shōgun,2024,10,TV Mini Series,['TV-14'],9.2,22000.0
2,3,True Detective,2014,31,TV Series,['A'],8.9,648000.0
3,4,The Walking Dead: The Ones Who Live,2024,6,TV Series,['TV-MA'],8.9,11000.0
4,5,One Day,2024,14,TV Mini Series,['TV-MA'],8.2,29000.0
...,...,...,...,...,...,...,...,...
95,96,Monk,2002,12,TV Series,['TV-PG'],8.1,84000.0
96,97,The Gentlemen,2024,8,TV Series,['TV-MA'],8.4,3000.0
97,98,Bridgerton,2020,25,TV Series,['18'],7.4,166000.0
98,99,Love Is Blind,2020,83,TV Series,['16'],6.2,11000.0


In [77]:
df['Grade'] = df['Grade'].apply(lambda x: extract_range(x, 2, -2))
df

,Rank,Show_name,Year,Episodes,Type,Grade,Rating_rate,Rating_count
0,1,Avatar: The Last Airbender,2024,10,TV Series,TV-PG,7.3,50000.0
1,2,Shōgun,2024,10,TV Mini Series,TV-14,9.2,22000.0
2,3,True Detective,2014,31,TV Series,A,8.9,648000.0
3,4,The Walking Dead: The Ones Who Live,2024,6,TV Series,TV-MA,8.9,11000.0
4,5,One Day,2024,14,TV Mini Series,TV-MA,8.2,29000.0
...,...,...,...,...,...,...,...,...
95,96,Monk,2002,12,TV Series,TV-PG,8.1,84000.0
96,97,The Gentlemen,2024,8,TV Series,TV-MA,8.4,3000.0
97,98,Bridgerton,2020,25,TV Series,18,7.4,166000.0
98,99,Love Is Blind,2020,83,TV Series,16,6.2,11000.0


In [90]:
df['Episodes'] = df.Episodes.astype('int')
df['Rating_rate'] = df.Rating_rate.astype('float')
df['Year'] = pd.to_datetime(df['Year'], format='%Y')
df['Year'] = pd.to_datetime(df['Year']).dt.year

In [95]:
df.to_csv('cleaned_imdb.csv')

In [1]:
pip install selenium

^C
Note: you may need to restart the kernel to use updated packages.
